<a href="https://colab.research.google.com/github/Mayur619/avc-detection/blob/notebooks/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import os
import cv2
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Conv2D,Activation,Input,Concatenate,Flatten,Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredLogarithmicError
from tensorflow.keras.applications import MobileNetV2,EfficientNetB0
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [81]:
DATA_PATH = "/content/drive/My Drive/AnimalVehicleCollision/data"
EPOCHS = 100
BATCH_SIZE = 8
ANIMAL_LABELS_DICT = {0:"cow",1:"dog",2:"bear"}
WIDTH=224
HEIGHT=224

In [82]:
class AnimalDataGenerator:
  def __init__(self,base_path,csv_path,batch_size):
    self.index=0
    self.base_path=base_path
    self.batch_size=batch_size
    self.df=pd.read_csv(csv_path)
  def __load_images(self,image_name_list,label_list):
    images=[]
    for i,image_name in enumerate(image_name_list):
      image=cv2.imread(os.path.join(self.base_path,ANIMAL_LABELS_DICT[label_list[i]],image_name))
      image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
      image=cv2.resize(image,(HEIGHT,WIDTH))
      images.append(image)
    return np.array(images)
  def get_steps_per_epoch(self):
    return self.df.shape[0]//self.batch_size
  def generate_image_batches(self,is_training):
    while True:
      for i in range(self.df.shape[0]//self.batch_size):
        bounding_boxes=self.df.iloc[self.index:self.index+self.batch_size,2:6].values
        animal_labels=self.df.iloc[self.index:self.index+self.batch_size,6].values
        images=self.__load_images(self.df.iloc[self.index:self.index+self.batch_size,1].values,animal_labels)
        animal_labels=to_categorical(animal_labels,num_classes=len(ANIMAL_LABELS_DICT.keys()))
        orientation=to_categorical(self.df.iloc[self.index:self.index+self.batch_size,-1].values,num_classes=4)
        path=self.df.iloc[self.index:self.index+self.batch_size,7:9].values
        yield [images,bounding_boxes,animal_labels],[orientation,path]
        self.index+=self.batch_size
        del animal_labels,bounding_boxes,images,orientation,path
      self.index=0

In [83]:
def cnn_model():
  input=Input(shape=(WIDTH,HEIGHT,3,),name="image_input")
  model=Conv2D(16,(3,3),padding='same')(input)
  model=Activation('relu')(model)
  model=Flatten()(model)
  model=Dense(16)(model)
  return input,model

In [84]:
def cnn_mobilenetv2():
  base_model=MobileNetV2(input_shape=(WIDTH,HEIGHT,3),weights='imagenet',include_top=False)
  for layer in base_model.layers:
    layer.trainable=False
  model=GlobalAveragePooling2D()(base_model.output)
  model=Activation('relu')(model)
  model=Flatten()(model)
  model=Dense(16)(model)
  return base_model.input, model

In [85]:
def cnn_efficientnetb0():
  base_model=EfficientNetB0(include_top=False,input_shape=(WIDTH,HEIGHT,3))
  for layer in base_model.layers:
    layer.trainable=False
  model=GlobalAveragePooling2D()(base_model.output)
  model=Activation('relu')(model)
  model=Flatten()(model)
  model=Dense(16)(model)
  return base_model.input, model

In [86]:
def bounding_box_model():
  input=Input(shape=(4,),name="bounding_box_input")
  model=Dense(8)(input)
  model=Activation('relu')(model)
  model=Flatten()(model)
  return input,model

In [87]:
def animal_label_model():
  input=Input(shape=(len(ANIMAL_LABELS_DICT.keys()),),name="animal_label_input")
  model=Dense(8)(input)
  model=Activation('relu')(model)
  model=Flatten()(model)
  return input,model

In [88]:
#cnn_input,cnn_output=cnn_model()
#cnn_input,cnn_output=cnn_mobilenetv2()
cnn_input,cnn_output=cnn_efficientnetb0()
bounding_box_input,bounding_box_output=bounding_box_model()
animal_label_input,animal_label_output=animal_label_model()
core_model=Concatenate(axis=1)([cnn_output,bounding_box_output,animal_label_output])
core_model=Dense(64)(core_model)
orientation=Dense(4,activation='softmax',name='orientation_output')(core_model)
path=Dense(2,name='path_output')(core_model)

In [89]:
model=Model(inputs=[cnn_input,bounding_box_input,animal_label_input],outputs=[orientation,path])

In [94]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss={
                  'orientation_output':CategoricalCrossentropy(from_logits=True),
                  'path_output':MeanSquaredLogarithmicError()
              })

In [95]:
train_generator=AnimalDataGenerator(os.path.join(DATA_PATH,"images"),os.path.join(DATA_PATH,"train.csv"),BATCH_SIZE)
test_generator=AnimalDataGenerator(os.path.join(DATA_PATH,"images"),os.path.join(DATA_PATH,"test.csv"),BATCH_SIZE)

In [96]:
model.fit(train_generator.generate_image_batches(True),
          epochs=3,
          steps_per_epoch=train_generator.get_steps_per_epoch(),
          validation_data=test_generator.generate_image_batches(True),
          validation_steps=test_generator.get_steps_per_epoch())

Epoch 1/3
421/421 [==============================] - 265s 628ms/step - loss: 1.1023 - orientation_output_loss: 1.0813 - path_output_loss: 0.0211 - val_loss: 1.1272 - val_orientation_output_loss: 1.1044 - val_path_output_loss: 0.0228
Epoch 2/3
421/421 [==============================] - 246s 584ms/step - loss: 1.0939 - orientation_output_loss: 1.0731 - path_output_loss: 0.0208 - val_loss: 1.1252 - val_orientation_output_loss: 1.1023 - val_path_output_loss: 0.0229
Epoch 3/3
421/421 [==============================] - 249s 593ms/step - loss: 1.0908 - orientation_output_loss: 1.0701 - path_output_loss: 0.0207 - val_loss: 1.1258 - val_orientation_output_loss: 1.1027 - val_path_output_loss: 0.0231
